## Chapter 5　応用

---
### 5.1　機械翻訳


#### 統計翻訳とニューラル翻訳
ある言語の文章を多言語の文章に翻訳する、人手を介さない方法論を機械翻訳（machine translation）と総称する。  
ルールベース翻訳、用例ベース翻訳、統計翻訳（statistical machine translation）などの方法論があり、  
系列変換モデルを利用した機械翻訳はニューラル翻訳（neural machine translation）と呼ばれる場合が多い。

---
#### NMTの典型的なモデル構成
[OpenNMT](http://opennmt.net/)（旧seq2seq-attn）が現在ベースラインと認識されるモデルの代表である。  
この中のモデルの１つについて取り上げる。

符号化器再帰層に２層bi-directional LSTMを使うとする。
$\Psi(\cdot)$ をLSTMの処理を表す関数とすると、擬似コードは次の通り。

\begin{align*}
&\text{Input: } \, X = (x_{i})_{i=1}^{I} \\
&\text{begin} \\
&\hspace{5pt}  h_{0}^{fw1} \leftarrow 0, h_{0}^{fw2} \leftarrow 0, h_{I+1}^{bw1} \leftarrow 0, h_{I+1}^{bw2} \leftarrow 0 \\
&\hspace{5pt}  \bar{X} \leftarrow E^{s}X \\
&\hspace{5pt}  \text{for} \, i \leftarrow 1 \, \text{to} \, I \, \text{do} \\
&\hspace{10pt} h_{i}^{fw1} \leftarrow \Psi^{fw1}(\bar{x}_{i},h_{i-1}^{fw1}) \\
&\hspace{10pt} h_{i}^{fw2} \leftarrow \Psi^{fw2}(h_{i}^{fw1},h_{i-1}^{fw2}) \\
&\hspace{5pt} \text{end} \\
&\hspace{5pt} \text{for} \, i \leftarrow 1 \, \text{to} \, I \, \text{do} \\
&\hspace{10pt} i' \leftarrow I+1-i \\
&\hspace{10pt} h_{i'}^{bw1} \leftarrow \Psi^{bw1}(\bar{x}_{i'},h_{i'+1}^{bw1}) \\
&\hspace{10pt} h_{i'}^{bw2} \leftarrow \Psi^{bw2}(h_{i'}^{bw1},h_{i'+1}^{bw2}) \\
&\hspace{10pt} h_{i'}^{s} = h_{i'}^{fw2} + h_{i'}^{bw2} \\
&\hspace{5pt} \text{end} \\
&\text{end} \\
&\text{Output: } \, H^{s}=(h_{i}^{s})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2} \\
\end{align*}



次に、復号化再帰層に２層LSTMを使うものとすると、擬似コードは次の通り。

評価時の復号化処理は一般にビーム探索を用いて処理を行うことが多い。
学習後に未知データを評価する際の復号化処理を擬似コードで表したものは次の通り。
（これはK=1のビーム探索と等価なアルゴリズムである。）